In [1]:
import pandas as pd
import os

# Pivot Input Data

In [2]:
# Read the deaths CSV file into a pandas DataFrame
deaths_df = pd.read_csv(os.getcwd() + '/Data/deaths.csv')

# Read the cases CSV file into a pandas DataFrame
cases_df = pd.read_csv(os.getcwd() + '/Data/cases.csv')

# Specify the column name of the county IDs
county_id_column = 'CountyID'

# Use the `melt` function to reshape the deaths DataFrame
melted_deaths_df = pd.melt(deaths_df, id_vars=county_id_column, var_name='Date', value_name='Deaths')

# Use the `melt` function to reshape the deaths DataFrame
melted_cases_df = pd.melt(cases_df, id_vars=county_id_column, var_name='Date', value_name='Cases')


melted_deaths_df.to_csv(os.getcwd() + "/Data/melted_deaths_df.csv", index=False)

melted_cases_df.to_csv(os.getcwd() + "/Data/melted_cases_df.csv", index=False)
print(melted_deaths_df, melted_cases_df)

         CountyID        Date  Deaths
0          1001.0  2020-03-01     0.0
1          1003.0  2020-03-01     0.0
2          1005.0  2020-03-01     0.0
3          1007.0  2020-03-01     0.0
4          1009.0  2020-03-01     0.0
...           ...         ...     ...
1026537   72151.0  2021-01-13     0.0
1026538   72153.0  2021-01-13     0.0
1026539   78010.0  2021-01-13     8.0
1026540   78020.0  2021-01-13     1.0
1026541   78030.0  2021-01-13    15.0

[1026542 rows x 3 columns]          CountyID        Date   Cases
0          1001.0  2020-03-01     0.0
1          1003.0  2020-03-01     0.0
2          1005.0  2020-03-01     0.0
3          1007.0  2020-03-01     0.0
4          1009.0  2020-03-01     0.0
...           ...         ...     ...
1026537   72151.0  2021-01-13   867.0
1026538   72153.0  2021-01-13   917.0
1026539   78010.0  2021-01-13   892.0
1026540   78020.0  2021-01-13   178.0
1026541   78030.0  2021-01-13  1096.0

[1026542 rows x 3 columns]


# Join Data

In [3]:
# Get the common columns between the two datasets
common_columns = list(set(melted_cases_df.columns) & set(melted_deaths_df.columns))

# Drop the common columns from df2 to avoid duplication
melted_deaths_df = melted_deaths_df.drop(common_columns, axis=1)

# Combine the datasets, preserving columns with the same name
combined_df = pd.concat([melted_cases_df, melted_deaths_df], axis=1)
combined_df

,CountyID,Date,Cases,Deaths
0,1001.0,2020-03-01,0.0,0.0
1,1003.0,2020-03-01,0.0,0.0
2,1005.0,2020-03-01,0.0,0.0
3,1007.0,2020-03-01,0.0,0.0
4,1009.0,2020-03-01,0.0,0.0
...,...,...,...,...
1026537,72151.0,2021-01-13,867.0,0.0
1026538,72153.0,2021-01-13,917.0,0.0
1026539,78010.0,2021-01-13,892.0,8.0
1026540,78020.0,2021-01-13,178.0,1.0


# Add State Data

In [4]:
import pandas as pd

def convert_county_to_state(county_fips):
    # Extract the first two digits to get the state FIPS code
    state_fip = str(county_fips)[:-5]
    state_fip = int(state_fip)

    return state_fip

# Add a new column 'StateName' based on 'CountyID' column
combined_df['StateID'] = combined_df['CountyID'].apply(convert_county_to_state).astype(int)

# Save the DataFrame with the added column to a new CSV file
combined_df.to_csv(os.getcwd() + "/Data/covid_df.csv", index=False)
combined_df

,CountyID,Date,Cases,Deaths,StateID
0,1001.0,2020-03-01,0.0,0.0,1
1,1003.0,2020-03-01,0.0,0.0,1
2,1005.0,2020-03-01,0.0,0.0,1
3,1007.0,2020-03-01,0.0,0.0,1
4,1009.0,2020-03-01,0.0,0.0,1
...,...,...,...,...,...
1026537,72151.0,2021-01-13,867.0,0.0,72
1026538,72153.0,2021-01-13,917.0,0.0,72
1026539,78010.0,2021-01-13,892.0,8.0,78
1026540,78020.0,2021-01-13,178.0,1.0,78


# Add Population

In [5]:
# Read the first XLSX file
population_df = pd.read_excel(os.getcwd() + '/Data/co-est2022-pop.xlsx')

# Read the second XLSX file
fips_df = pd.read_excel(os.getcwd() + '/Data/all-geocodes-v2022.xlsx')

# Drop unnecessary columns from fips_df
fips_df = fips_df.drop(['Consolidated City FIPS Code', 'County Subdivision FIPS Code', 'Place FIPS Code', 'Summary Level'], axis=1)
fips_df

,State FIPS Code,County FIPS Code,Area Name
0,0,0,United States
1,1,0,Alabama
2,1,1,Autauga County
3,1,3,Baldwin County
4,1,5,Barbour County
...,...,...,...
43813,72,145,Vega Baja Municipio
43814,72,147,Vieques Municipio
43815,72,149,Villalba Municipio
43816,72,151,Yabucoa Municipio


In [6]:
# Split the "Area Name" column into two columns
population_df[['County Name', 'State Name']] = population_df['Area Name'].str.split(',', expand=True)
population_df["County Name"] = population_df["County Name"].str.strip()
population_df["County Name"] = population_df["County Name"].str.replace(".", "")
population_df["State Name"] = population_df["State Name"].str.strip()
population_df.dropna()
# Rename column to make sense
merged_df = population_df.rename(columns={2021: '2021 Population'})
population_df = population_df.drop(['Area Name'], axis=1)

population_df

,2021,County Name,State Name
0,332031554,United States,None
1,59210,Autauga County,Alabama
2,239361,Baldwin County,Alabama
3,24539,Barbour County,Alabama
4,22370,Bibb County,Alabama
...,...,...,...
3140,41582,Sweetwater County,Wyoming
3141,23622,Teton County,Wyoming
3142,20655,Uinta County,Wyoming
3143,7712,Washakie County,Wyoming


In [7]:
# Merge "State Name" into population_df
fips_df = fips_df.rename(columns={'Area Name': 'State Name'})
population_df = pd.merge(population_df, fips_df[["State FIPS Code", "State Name"]], on='State Name', how='left')


# Merge "County FIPS Code" into population_df
fips_df = fips_df.rename(columns={'State Name': 'County Name'})
population_df = pd.merge(population_df, fips_df[['State FIPS Code', 'County Name', 'County FIPS Code']], on=['State FIPS Code', 'County Name'], how='left')

# Create Complete FIPS Column
population_df = population_df.dropna(subset=['County FIPS Code', 'State FIPS Code'])
population_df['County FIPS Code'] = population_df['County FIPS Code'].astype(int).astype(str)
population_df['County FIPS Code'] = population_df['County FIPS Code'].str.zfill(3)
population_df['State FIPS Code'] = population_df['State FIPS Code'].astype(int).astype(str)
population_df['CountyID'] = population_df['State FIPS Code'] + population_df['County FIPS Code']
population_df['CountyID'] = population_df['CountyID'].astype(int)
population_df = population_df.drop(['State FIPS Code', 'County FIPS Code'], axis=1)

population_df.to_csv(os.getcwd() + '/Data/population_df.csv', index=False)


# Join and Format Population + Covid

In [8]:
# Join merged_df with combined_df
population_df = population_df.rename(columns={'FIPS Code': 'CountyID'})
joined_df = pd.merge(combined_df, population_df, on='CountyID', how='left')

joined_df = joined_df.rename(columns={2021: '2021 Population'})
joined_df = joined_df.reindex(columns=['Date', 'CountyID', 'StateID', 'State Name', 'Cases', 'Deaths', '2021 Population'])
joined_df = joined_df.dropna(subset=['Date', 'CountyID', 'StateID', 'State Name', 'Cases', 'Deaths', '2021 Population'])

print(joined_df)


               Date  CountyID  StateID State Name   Cases  Deaths  \
0        2020-03-01    1001.0        1    Alabama     0.0     0.0   
1        2020-03-01    1003.0        1    Alabama     0.0     0.0   
2        2020-03-01    1005.0        1    Alabama     0.0     0.0   
3        2020-03-01    1007.0        1    Alabama     0.0     0.0   
4        2020-03-01    1009.0        1    Alabama     0.0     0.0   
...             ...       ...      ...        ...     ...     ...   
1026773  2021-01-13   56037.0       56    Wyoming  3222.0    26.0   
1026774  2021-01-13   56039.0       56    Wyoming  2502.0     4.0   
1026775  2021-01-13   56041.0       56    Wyoming  1774.0     9.0   
1026776  2021-01-13   56043.0       56    Wyoming   829.0    23.0   
1026777  2021-01-13   56045.0       56    Wyoming   601.0     4.0   

         2021 Population  
0                59210.0  
1               239361.0  
2                24539.0  
3                22370.0  
4                59085.0  
...      

# Add Change in Deaths/Cases Columns

In [9]:
cases_counts = {}
change_in_cases = []
for index, row in joined_df.iterrows():
    countyID = row['CountyID']
    cases = row['Cases']
    if countyID not in cases_counts:
        change_in_cases.append(cases)
    else:
        change_in_cases.append(cases - cases_counts[countyID])
    cases_counts[countyID] = cases

In [10]:
deaths_counts = {}
change_in_deaths = []
for index, row in joined_df.iterrows():
    countyID = row['CountyID']
    deaths = row['Deaths']
    if countyID not in deaths_counts:
        change_in_deaths.append(deaths)
    else:
        change_in_deaths.append(deaths - deaths_counts[countyID])
    deaths_counts[countyID] = deaths

In [11]:
joined_df['Cases Delta'] = change_in_cases
joined_df['Deaths Delta'] = change_in_deaths

In [12]:
joined_df = joined_df.dropna()
joined_df.to_csv(os.getcwd() + '/Data/joined_df.csv', index=False)